# Test 02_process_variable_ds.py interactively

In [16]:
import numpy as np
import xarray as xr
from pathlib import Path
from datetime import datetime
from scipy.ndimage import distance_transform_edt

import idd_climate_models.constants as rfc
from idd_climate_models.climate_file_functions import should_regrid, regrid_with_cdo
from idd_climate_models.validation_functions import is_monthly

In [39]:
# === PARAMETERS ===
data_source = 'cmip6'
model = 'MRI-ESM2-0'
variant = 'r4i1p1f1'
scenario = 'historical'
time_period = '1970-1980'
variable = 'ua'  # Start with monthly, change to 'ua' for daily test
grid = 'gn'
frequency = 'day'  # Change to 'day' for ua/va

start_year, end_year = map(int, time_period.split('-'))
print(f"Processing {variable} for {model}/{variant}/{scenario}, {start_year}-{end_year}")

Processing ua for MRI-ESM2-0/r4i1p1f1/historical, 1970-1980


## Step 1: Find overlapping files

In [40]:
import re

def parse_date_from_filename(filename, frequency):
    """Extract start and end dates from a NetCDF filename."""
    if 'day' in frequency:
        match = re.search(r'_(\d{8})-(\d{8})\.nc$', filename)
        if match:
            return int(match.group(1)[:4]), int(match.group(2)[:4])
    elif 'mon' in frequency.lower() or 'Amon' in frequency or 'Omon' in frequency:
        match = re.search(r'_(\d{6})-(\d{6})\.nc$', filename)
        if match:
            return int(match.group(1)[:4]), int(match.group(2)[:4])
    return None, None

def find_overlapping_files(raw_dir, start_year, end_year, frequency):
    """Find all raw files that overlap with [start_year, end_year]."""
    if not raw_dir.exists():
        return []
    overlapping = []
    for f in sorted(raw_dir.glob("*.nc")):
        file_start, file_end = parse_date_from_filename(f.name, frequency)
        if file_start is None:
            print(f"Warning: Could not parse dates from {f.name}")
            overlapping.append(f)
            continue
        if file_end >= start_year and file_start <= end_year:
            overlapping.append(f)
    return overlapping

In [42]:
raw_dir = rfc.RAW_DATA_PATH / data_source / model / variant / scenario / variable / grid / frequency
print(f"Raw dir: {raw_dir}")
print(f"Exists: {raw_dir.exists()}")

raw_files = find_overlapping_files(raw_dir, start_year, end_year, frequency)
print(f"\nFound {len(raw_files)} overlapping files:")
for f in raw_files:
    print(f"  - {f.name}")

Raw dir: /mnt/team/rapidresponse/pub/tropical-storms/data/raw/cmip6/MRI-ESM2-0/r4i1p1f1/historical/ua/gn/day
Exists: True

Found 2 overlapping files:
  - ua_day_MRI-ESM2-0_historical_r4i1p1f1_gn_19700101-19791231.nc
  - ua_day_MRI-ESM2-0_historical_r4i1p1f1_gn_19800101-19891231.nc


## Step 2: Load and clip data

In [43]:
%%time
print("Loading data (lazy)...")
ds = xr.open_mfdataset(
    raw_files,
    combine='by_coords',
    engine='netcdf4'
)
print(f"Loaded: {len(ds.time)} timesteps")
print(f"Time range: {ds.time.values[0]} to {ds.time.values[-1]}")
print(f"variables: {list(ds.data_vars)}")
print(f"Dims: {dict(ds.dims)}")

Loading data (lazy)...
Loaded: 7305 timesteps
Time range: 1970-01-01T12:00:00.000000000 to 1989-12-31T12:00:00.000000000
variables: ['time_bnds', 'lat_bnds', 'lon_bnds', 'ua']
Dims: {'time': 7305, 'bnds': 2, 'lat': 160, 'lon': 320, 'plev': 8}
CPU times: user 199 ms, sys: 138 ms, total: 337 ms
Wall time: 19.8 s


<timed exec>:2: FutureWarning: In a future version of xarray the default value for data_vars will change from data_vars='all' to data_vars=None. This is likely to lead to different results when multiple datasets have matching variables with overlapping values. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set data_vars explicitly.
<timed exec>:10: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.


In [44]:
%%time
print(f"Clipping to {start_year}-{end_year}...")
ds = ds.sel(time=slice(f"{start_year}-01-01", f"{end_year}-12-31"))
actual_years = sorted(set(ds.time.dt.year.values))
print(f"After clipping: {len(ds.time)} timesteps, years {actual_years[0]}-{actual_years[-1]}")

Clipping to 1970-1980...
After clipping: 4018 timesteps, years 1970-1980
CPU times: user 53.7 ms, sys: 3.65 ms, total: 57.3 ms
Wall time: 58.9 ms


## Step 3: Load into memory

In [22]:
%%time
print("Loading into memory...")
ds = ds.compute()
print("Done!")
print(f"Memory size estimate: {ds.nbytes / 1e9:.2f} GB")

Loading into memory...
Done!
Memory size estimate: 0.09 GB
CPU times: user 1.99 s, sys: 267 ms, total: 2.26 s
Wall time: 3.66 s


In [23]:
ds

<xarray.Dataset> Size: 95MB
Dimensions:             (time: 204, bnds: 2, i: 292, j: 362, vertices: 4)
Coordinates:
  * time                (time) object 2kB 1970-01-16 12:00:00 ... 1986-12-16 ...
  * i                   (i) int32 1kB 0 1 2 3 4 5 6 ... 286 287 288 289 290 291
  * j                   (j) int32 1kB 0 1 2 3 4 5 6 ... 356 357 358 359 360 361
    latitude            (i, j) float64 846kB -78.39 -78.39 ... 50.01 50.01
    longitude           (i, j) float64 846kB 72.5 73.5 74.5 ... 73.04 73.0 72.99
Dimensions without coordinates: bnds, vertices
Data variables:
    time_bnds           (time, bnds) object 3kB 1970-01-01 00:00:00 ... 1987-...
    vertices_latitude   (i, j, vertices) float64 3MB -78.39 -78.39 ... 49.98
    vertices_longitude  (i, j, vertices) float64 3MB 72.5 72.5 ... 72.95 73.0
    tos                 (time, i, j) float32 86MB nan nan nan ... nan nan nan
Attributes: (12/48)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    comment:                none
    ...                     ...
    title:                  CMCC-ESM2 output prepared for CMIP6
    variable_id:            tos
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by CMCC is licensed und...
    cmor_version:           3.6.0
    tracking_id:            hdl:21.14100/688e38fa-19d8-4550-afdd-18f161fa2fdb

## Step 4: Check for NaNs

In [24]:
print("Checking for NaNs...")
for var in ds.data_vars:
    nan_count = ds[var].isnull().sum().values
    total = ds[var].size
    pct = 100 * nan_count / total
    print(f"  {var}: {nan_count:,} NaNs ({pct:.2f}%)")

Checking for NaNs...
  time_bnds: 0 NaNs (0.00%)
  vertices_latitude: 0 NaNs (0.00%)
  vertices_longitude: 0 NaNs (0.00%)
  tos: 8,250,168 NaNs (38.26%)


## Step 5: Fill NaNs (if needed)

In [25]:
def fill_nans_nearest(data):
    """Fills NaNs using nearest neighbor interpolation."""
    if not np.any(np.isnan(data)):
        return data
    
    data_filled = data.copy()
    
    if data.ndim == 2:
        mask = np.isnan(data)
        ind = distance_transform_edt(mask, return_distances=False, return_indices=True)
        data_filled[mask] = data[tuple(ind[:, mask])]
        return data_filled
    
    # 3D+ arrays: check if mask is constant
    first_mask = np.isnan(data[0, ...])
    mask_is_constant = all(
        np.array_equal(first_mask, np.isnan(data[t, ...]))
        for t in range(min(5, data.shape[0]))
    )
    
    if mask_is_constant and np.any(first_mask):
        print("  NaN mask is constant - using optimized single-index computation")
        ind = distance_transform_edt(first_mask, return_distances=False, return_indices=True)
        for t in range(data.shape[0]):
            data_filled[t, ...][first_mask] = data[t, ...][tuple(ind[:, first_mask])]
    else:
        print(f"  NaN mask varies - processing {data.shape[0]} timesteps")
        for t in range(data.shape[0]):
            slice_data = data[t, ...]
            mask = np.isnan(slice_data)
            if np.any(mask):
                ind = distance_transform_edt(mask, return_distances=False, return_indices=True)
                data_filled[t, ...][mask] = slice_data[tuple(ind[:, mask])]
    
    return data_filled

In [26]:
%%time
has_nans = any(ds[var].isnull().any() for var in ds.data_vars)

if has_nans:
    print("Filling NaNs...")
    ds_filled = xr.Dataset(coords=ds.coords, attrs=ds.attrs)
    for var_name in ds.data_vars:
        variable_ds = ds[var_name]
        if variable_ds.isnull().any():
            print(f"  Filling {var_name}...")
            filled_np_data = fill_nans_nearest(variable_ds.values)
            ds_filled[var_name] = xr.DataArray(
                filled_np_data, coords=variable_ds.coords, dims=variable_ds.dims, attrs=variable_ds.attrs
            )
        else:
            ds_filled[var_name] = variable_ds
else:
    print("No NaNs found.")
    ds_filled = ds

print("Done!")

Filling NaNs...
  Filling tos...
  NaN mask is constant - using optimized single-index computation
Done!
CPU times: user 726 ms, sys: 43.8 ms, total: 770 ms
Wall time: 765 ms


## Step 6: Check regridding & save

In [27]:
TARGET_GRID = 'r360x180'
needs_regridding = should_regrid(str(raw_files[0]), target_grid=TARGET_GRID)
output_grid = 'gr' if needs_regridding else grid

print(f"Needs regridding: {needs_regridding}")
print(f"Output grid: {output_grid}")

Needs regridding: True
Output grid: gr


In [29]:
# Construct output path
target_dir = rfc.TC_RISK_INPUT_PATH / data_source / model / variant / scenario / time_period
target_dir.mkdir(parents=True, exist_ok=True)

if is_monthly(frequency):
    date_start = f"{start_year}01"
    date_end = f"{end_year}12"
else:
    date_start = f"{start_year}0101"
    date_end = f"{end_year}1231"

output_filename = f"{variable}_{frequency}_{model}_{scenario}_{variant}_{output_grid}_{date_start}-{date_end}.nc"
output_path = target_dir / output_filename

print(f"Output: {output_path}")

Output: /mnt/team/rapidresponse/pub/tropical-storms/tc_risk/input/cmip6/CMCC-ESM2/r1i1p1f1/historical/1970-1986/tos_Omon_CMCC-ESM2_historical_r1i1p1f1_gr_197001-198612.nc


In [30]:
total_points = ds_filled['tos'].size
valid_points = ds_filled['tos'].notnull().sum().values

print(f"Total points: {total_points:,}")
print(f"Valid points: {valid_points:,}")
print(f"Percent data: {(valid_points/total_points)*100:.2f}%")

Total points: 21,563,616
Valid points: 21,563,616
Percent data: 100.00%


In [32]:
# This will return the min, max, and mean while ignoring NaNs
# If the result is 'nan', then the whole array is indeed empty.
print(f"Min:  {ds_filled['tos'].min().values}")
print(f"Max:  {ds_filled['tos'].max().values}")
print(f"Mean: {ds_filled['tos'].mean().values}")

Min:  -1.7999999523162842
Max:  33.97351837158203
Mean: 12.278160095214844


In [34]:
445*11

4895

In [33]:
ds_filled

<xarray.Dataset> Size: 95MB
Dimensions:             (time: 204, i: 292, j: 362, bnds: 2, vertices: 4)
Coordinates:
  * time                (time) object 2kB 1970-01-16 12:00:00 ... 1986-12-16 ...
  * i                   (i) int32 1kB 0 1 2 3 4 5 6 ... 286 287 288 289 290 291
  * j                   (j) int32 1kB 0 1 2 3 4 5 6 ... 356 357 358 359 360 361
    latitude            (i, j) float64 846kB -78.39 -78.39 ... 50.01 50.01
    longitude           (i, j) float64 846kB 72.5 73.5 74.5 ... 73.04 73.0 72.99
Dimensions without coordinates: bnds, vertices
Data variables:
    time_bnds           (time, bnds) object 3kB 1970-01-01 00:00:00 ... 1987-...
    vertices_latitude   (i, j, vertices) float64 3MB -78.39 -78.39 ... 49.98
    vertices_longitude  (i, j, vertices) float64 3MB 72.5 72.5 ... 72.95 73.0
    tos                 (time, i, j) float32 86MB 4.134 4.134 ... -1.799 -1.799
Attributes: (12/48)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    comment:                none
    ...                     ...
    title:                  CMCC-ESM2 output prepared for CMIP6
    variable_id:            tos
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by CMCC is licensed und...
    cmor_version:           3.6.0
    tracking_id:            hdl:21.14100/688e38fa-19d8-4550-afdd-18f161fa2fdb

In [ ]:
# Optional: uncomment to actually save
# if 'time_bnds' in ds_filled:
#     ds_filled = ds_filled.drop_vars('time_bnds')
# 
# encoding = {var: {'zlib': True, 'complevel': 7, 'shuffle': True} for var in ds_filled.data_vars}
# ds_filled.to_netcdf(output_path, encoding=encoding, engine='netcdf4')
# print(f"Saved to {output_path}")

In [45]:
import xarray as xr

# Test one of the failing files
test_file = "/mnt/team/rapidresponse/pub/tropical-storms/tc_risk/input/cmip6/CMCC-ESM2/r1i1p1f1/historical/1970-1986/ua_day_CMCC-ESM2_historical_r1i1p1f1_gr_19700101-19861231.nc"

ds = xr.open_dataset(test_file)

# Check time encoding
print("Time coordinate dtype:", ds.time.dtype)
print("Time encoding:", ds.time.encoding)
print("Time values (first 5):", ds.time.values[:5])
print("\nTime attributes:", ds.time.attrs)

# Try to access what TC-risk might be checking
try:
    time_values = ds['time'].values
    print(f"\nTime type: {type(time_values[0])}")
    print(f"Time calendar: {ds.time.encoding.get('calendar', 'NOT SET')}")
    print(f"Time units: {ds.time.encoding.get('units', 'NOT SET')}")
except Exception as e:
    print(f"\nError accessing time: {e}")

ds.close()

Time coordinate dtype: object
Time encoding: {'dtype': dtype('float64'), 'zlib': False, 'szip': False, 'zstd': False, 'bzip2': False, 'blosc': False, 'shuffle': False, 'complevel': 0, 'fletcher32': False, 'contiguous': False, 'chunksizes': (512,), 'preferred_chunks': {'time': 512}, 'source': '/mnt/team/rapidresponse/pub/tropical-storms/tc_risk/input/cmip6/CMCC-ESM2/r1i1p1f1/historical/1970-1986/ua_day_CMCC-ESM2_historical_r1i1p1f1_gr_19700101-19861231.nc', 'original_shape': (6205,), 'units': 'days since 1850-01-01', 'calendar': '365_day'}
Time values (first 5): [cftime.DatetimeNoLeap(1970, 1, 1, 12, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeNoLeap(1970, 1, 2, 12, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeNoLeap(1970, 1, 3, 12, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeNoLeap(1970, 1, 4, 12, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeNoLeap(1970, 1, 5, 12, 0, 0, 0, has_year_zero=True)]

Time attributes: {'standard_name': 'time', 'long_name': 'time', 'bounds': 'time_bnds', 'ax